<a href="https://colab.research.google.com/github/anushkasingh2002/ML-Assignments/blob/main/ML_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv('/abalone_data.csv')

In [ ]:
df.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [ ]:
# Convert 'Sex' to numerical values
df['Sex'] = df['Sex'].map({'M': 0, 'F': 1, 'I': 2})

# Perform one-hot encoding for 'Sex' column
sex_one_hot = pd.get_dummies(df['Sex'], prefix='Sex')
df = pd.concat([df, sex_one_hot], axis=1)
df = df.drop(['Sex'], axis=1)  # Drop the original 'Sex' column

# Split the data into features (X) and target (y)
X = df.drop(['Rings'], axis=1).values
y = df['Rings'].values


In [ ]:
df.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Sex_0,Sex_1,Sex_2
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,True,False,False
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,True,False,False
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,False,True,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,True,False,False
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,False,False,True


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)


##Converting to Pytorch tensors

In [ ]:

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
class AgeClassifier(nn.Module):
    def __init__(self):
        super(AgeClassifier, self).__init__()
        self.fc1 = nn.Linear(10, 64)  # Input layer with 10 nodes
        self.relu1 = nn.ReLU()  # ReLU activation for first hidden layer
        self.fc2 = nn.Linear(64, 32)  # First hidden layer with 64 nodes
        self.relu2 = nn.ReLU()  # ReLU activation for second hidden layer
        self.fc3 = nn.Linear(32, 1)  # Output layer with 1 node for regression

    def forward(self, x):
        x = self.relu1(self.fc1(x))  # Pass through first hidden layer with ReLU activation
        x = self.relu2(self.fc2(x))  # Pass through second hidden layer with ReLU activation
        x = self.fc3(x)  # Output layer
        return x

# Create an instance of the AgeClassifier
model = AgeClassifier()

In [ ]:
import torch.optim as optim

# Define the Mean Squared Error (MSE) loss function
criterion = nn.MSELoss()

# Define the Stochastic Gradient Descent (SGD) optimizer with learning rate 0.1
optimizer = optim.SGD(model.parameters(), lr=0.1)

#Model Evaluation

In [ ]:
# Train the model for 100 epochs
epochs = 100
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.unsqueeze(1))

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

# Evaluation function
def evaluate(model, X_test, y_test):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(X_test)
        mse = nn.MSELoss()
        loss = mse(outputs, y_test.unsqueeze(1))
        return loss.item()

# Evaluate the model on the testing set
test_loss = evaluate(model, X_test_tensor, y_test_tensor)
print(f'Test Loss (MSE): {test_loss}')

Epoch 1/100, Loss: 109.23081970214844
Epoch 11/100, Loss: 30.907306671142578
Epoch 21/100, Loss: 10.260530471801758
Epoch 31/100, Loss: 8.212692260742188
Epoch 41/100, Loss: 7.276365280151367
Epoch 51/100, Loss: 7.13868522644043
Epoch 61/100, Loss: 7.8527655601501465
Epoch 71/100, Loss: 7.556180953979492
Epoch 81/100, Loss: 7.284013748168945
Epoch 91/100, Loss: 7.138354301452637
Test Loss (MSE): 7.142284870147705


##Hyperparameter Tunining

In [ ]:
import torch.nn.functional as F

class AgeClassifier(nn.Module):
    def __init__(self, input_size, hidden_nodes, num_layers):
        super(AgeClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_nodes)  # Input layer with specified number of nodes
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_nodes, hidden_nodes) for _ in range(num_layers)])
        self.fc3 = nn.Linear(hidden_nodes, 1)  # Output layer with 1 node for regression

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Pass through first hidden layer with ReLU activation
        for layer in self.hidden_layers:
            x = F.relu(layer(x))  # Pass through hidden layers with ReLU activation
        x = self.fc3(x)  # Output layer
        return x

# Hyperparameter tuning loop
for lr in learning_rates:
    for batch_size in batch_sizes:
        for nodes in hidden_nodes:
            for num_layers in num_layers_list:
                # Create model instance
                model = AgeClassifier(input_size=X_train_tensor.shape[1], hidden_nodes=nodes, num_layers=num_layers)

                # Define optimizer with current learning rate
                optimizer = optim.SGD(model.parameters(), lr=lr)

                # Train the model
                for epoch in range(epochs):
                    model.train()
                    optimizer.zero_grad()
                    outputs = model(X_train_tensor)
                    loss = criterion(outputs, y_train_tensor.unsqueeze(1))
                    loss.backward()
                    optimizer.step()

                # Evaluate the model
                model.eval()
                with torch.no_grad():
                    outputs = model(X_test_tensor)
                    mse = criterion(outputs, y_test_tensor.unsqueeze(1))

                # Store results in dictionary
                results[(lr, batch_size, nodes, num_layers)] = mse.item()

# Find the hyperparameters with the lowest MSE
best_hyperparameters = min(results, key=results.get)
best_mse = results[best_hyperparameters]

print(f"Best Hyperparameters: Learning Rate={best_hyperparameters[0]}, Batch Size={best_hyperparameters[1]}, Hidden Nodes={best_hyperparameters[2]}, Number of Layers={best_hyperparameters[3]}")
print(f"Best MSE: {best_mse}")


Best Hyperparameters: Learning Rate=0.01, Batch Size=64, Hidden Nodes=32, Number of Layers=1
Best MSE: 6.569011688232422


##Adagrad Optimizer

In [ ]:
# Define hyperparameters
learning_rate = 0.01
batch_size = 64
hidden_nodes = 32
num_layers = 3  # Increase the number of layers to 3

# Create model instance with increased number of layers
model = AgeClassifier(input_size=X_train_tensor.shape[1], hidden_nodes=hidden_nodes, num_layers=num_layers)

# Define the Adagrad optimizer with specified learning rate
optimizer_adagrad = optim.Adagrad(model.parameters(), lr=learning_rate)

# Training loop with Adagrad optimizer
for epoch in range(epochs):
    model.train()
    optimizer_adagrad.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.unsqueeze(1))
    loss.backward()
    optimizer_adagrad.step()

# Evaluate the model with Adagrad optimizer
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    mse_adagrad = criterion(outputs, y_test_tensor.unsqueeze(1))

# Compare with SGD optimizer
optimizer_sgd = optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(epochs):
    model.train()
    optimizer_sgd.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.unsqueeze(1))
    loss.backward()
    optimizer_sgd.step()

# Evaluate the model with SGD optimizer
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    mse_sgd = criterion(outputs, y_test_tensor.unsqueeze(1))

print(f"MSE with Adagrad optimizer: {mse_adagrad.item()}")
print(f"MSE with SGD optimizer: {mse_sgd.item()}")

MSE with Adagrad optimizer: 6.88235330581665
MSE with SGD optimizer: 7.616939067840576


##Sigmoid and 10 hidden layers

In [ ]:
import torch.optim as optim

# Define hyperparameters
learning_rate = 0.01
batch_size = 64
hidden_nodes = 32
num_layers = 10  # Increase the number of hidden layers to 10

# Create model instance with 10 hidden layers and Sigmoid activation
class AgeClassifier(nn.Module):
    def __init__(self, input_size, hidden_nodes, num_layers):
        super(AgeClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_nodes)  # Input layer with specified number of nodes
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_nodes, hidden_nodes) for _ in range(num_layers)])
        self.fc3 = nn.Linear(hidden_nodes, 1)  # Output layer with 1 node for regression
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))  # Pass through first hidden layer with Sigmoid activation
        for layer in self.hidden_layers:
            x = self.sigmoid(layer(x))  # Pass through hidden layers with Sigmoid activation
        x = self.fc3(x)  # Output layer
        return x

# Create model instance with increased number of layers and Sigmoid activation
model = AgeClassifier(input_size=X_train_tensor.shape[1], hidden_nodes=hidden_nodes, num_layers=num_layers)

# Define the SGD optimizer with specified learning rate
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.unsqueeze(1))
    loss.backward()
    optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    mse = criterion(outputs, y_test_tensor.unsqueeze(1))

print(f"MSE with 10 hidden layers and Sigmoid activation: {mse.item()}")


MSE with 10 hidden layers and Sigmoid activation: 10.828198432922363
